# This notebook shows how a SuloClassifier beat a single model as well as a Voting classifier due to its superior design

### We are going to test it on a large dataset using the sample example code provided by:
https://machinelearningmastery.com/weighted-average-ensemble-with-python/
Thanks to Jason Brownlee for his Machine Learning Mastery blogs. He is absolutely great!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from featurewiz import SuloClassifier

C:\Users\Ram\AppData\Roaming\Python\Python38\site-packages\xgboost\compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Imported version = 0.1.86.
from featurewiz import FeatureWiz
wiz = FeatureWiz(verbose=1)
X_train_selected = wiz.fit_transform(X_train, y_train)
X_test_selected = wiz.transform(X_test)
wiz.features  ### provides a list of selected features ###
                                


In [3]:
# evaluate a weighted average ensemble for classification compared to base model
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier

# get a list of base models
lgbm = LGBMClassifier(random_state=0, n_estimators=100)
def get_models():
	models = list()
	models.append(('lr', LogisticRegression()))
	models.append(('LGBM', lgbm))
	models.append(('bayes', GaussianNB()))
	return models
 
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores
 
# define dataset
X, y = make_classification(n_samples=100000, n_features=50, n_informative=40, 
                           n_redundant=5, random_state=7)
# split dataset into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.50, random_state=1)
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.33, random_state=1)
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)
# create the ensemble
#ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
ensemble = SuloClassifier(base_estimator=None, n_estimators=None, verbose=1)
# fit the ensemble on the training dataset
ensemble.fit(pd.DataFrame(X_train_full), pd.Series(y_train_full))
# make predictions on test set
yhat = ensemble.predict(X_test)
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('SuloClassifier Accuracy: %.3f' % (score*100))
# evaluate each standalone model
scores = evaluate_models(models, X_train_full, X_test, y_train_full, y_test)
for i in range(len(models)):
	print('>%s: %.3f' % (models[i][0], scores[i]*100))
# evaluate equal weighting
ensemble = VotingClassifier(estimators=models, voting='soft')
ensemble.fit(X_train_full, y_train_full)
yhat = ensemble.predict(X_test)
score = accuracy_score(y_test, yhat)
print('Voting Accuracy: %.3f' % (score*100))

[0.8548484848484849, 0.9620606060606061, 0.8383030303030303]
Input data shapes: X = (50000, 50), y = (50000,)
No GPU available on this device. Using CPU for lightgbm and others.
Number of estimators = 7
    Number of estimators used in SuloClassifier = 7
    Selecting LGBM Classifier for this dataset...
Finding best params for base estimator using RandomizedSearchCV...
    best score is : 0.9841348133071002
    best Params is : {'model__learning_rate': 0.4120463984051167, 'model__n_estimators': 179}
Time Taken for RandomizedSearchCV: 85 (seconds)
Bal accu 98%
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3597
           1       0.99      0.98      0.98      3546

    accuracy                           0.99      7143
   macro avg       0.99      0.98      0.99      7143
weighted avg       0.99      0.99      0.99      7143

    Fold 1: OOF Score: 98%
Bal accu 98%
              precision    recall  f1-score   support

           0